# Train Ad-Hoc Model in this Notebook

In [ ]:
!pip install -q boto3
!pip install -q xgboost==0.90

In [ ]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

# Load the data

In [ ]:
import glob
import pandas as pd
import os

def load_dataset(path, sep, header):
    if (os.path.isdir(path)):
        data = pd.concat([pd.read_csv(f, sep=sep, header=header) for f in glob.glob('{}/*.csv'.format(path))], ignore_index = True)
    else:
        data = pd.concat([pd.read_csv(f, sep=sep, header=header) for f in glob.glob('{}'.format(path))], ignore_index = True)        

    labels = data.iloc[:,0]
    features = data.drop(data.columns[0], axis=1)
    
    if header==None:
        # Adjust the column names after dropping the 0th column above
        # New column names are 0 (inclusive) to len(features.columns) (exclusive)
        new_column_names = list(range(0, len(features.columns)))
        features.columns = new_column_names

    return features, labels

In [ ]:
!ls -al ./data/train.csv

In [ ]:
X_train, y_train = load_dataset(path='./data/train.csv', sep=',', header=0)

In [ ]:
X_train.shape

In [ ]:
X_train.head(5)

# Transform the raw text into TF/IDF

Before we train our XGBoost model, we need to convert our raw review text features into numeric features.  

To do this, we use a technique called `Term Frequency Inverse Document Frequency` or `TF/IDF`.  

Here is the Wikipedia definition:  https://en.wikipedia.org/wiki/Tf%E2%80%93idf

Common words and terms are naturally weighted lower because of the Inverse Document Frequency portion of TF/IDF.  The end result is that we represent each review as a vector of floats versus text-based words.

TF/IDF will creates vectors with 1,000's of values for each text-based review... 1 value for each term or word.  Therefore, we must reduce our dimensions using a technique called `Singular Value Decomposition` or `SVD`.  Here is the Wikipedia definition:  https://en.wikipedia.org/wiki/Singular_value_decomposition.

In Natural Language Processing (NLP) context, the combination of TF/IDF and SVD is typically called `Latent Semantic Analysis` or `LSA`:  https://en.wikipedia.org/wiki/Latent_semantic_analysis.  Every modern NLP library supports TF/IDF, SVD, and LSA.

Note that we have to manually chose a set of hyper-parameters for TF/IDF including `max_df`, `min_df`, `max_features`, etc.  We also need to specify hyper-parameters for `TruncatedSVD` including `num_components`.  AutoPilot will automatically chose these hyper-parameters for us.  But, for now, we must chose them ourselves based on intuition, experience, and sometimes luck!

Lastly, we scale our features to squash outliers by centering and scaling our data around a normal Gaussian distribution with mean 0 and unit variance.  If we don't perform this scaling, features with large variance may dominate and prevent our model from fitting to our dataset.

Altogether, we create a `pipeline` of feature transformations.  Note that this entire pipeline must be applied to our features during both model training *and* model predicting.  This highlights one of the key difficulties of `ad-hoc` model training like we are doing here.  How do I re-use this pipeline outside of this notebook?  We will cover this soon.

### Define our transformation function

In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

def feature_transform_fn(df_text, column_name):
    text_processors = Pipeline(
        steps=[
            (
                'tfidfvectorizer',
                TfidfVectorizer(
                    max_df=0.25,                                       
                    min_df=.0025,
                    analyzer='word',
                    max_features=10000
                )
            )
        ]
    )

    column_transformer = ColumnTransformer(
        transformers=[('text_processing', text_processors, df_text.columns.get_loc(column_name))]
    )

    pipeline = Pipeline(
        steps=[
            ('column_transformer', column_transformer), 
            ('dimension_reducer', TruncatedSVD(n_components=300)),
            ('standard_scaler', StandardScaler())
        ]
    )

    return pipeline

### Transform our raw text into vectors
_This will take a minute or two.  Please be patient._

In [ ]:
np_tfidf = feature_transform_fn(df_text=X_train, column_name='review_body').fit_transform(X_train)
df_tfidf = pd.DataFrame(np_tfidf)
df_tfidf.shape

# Train the Model using XGBoost and our New TF/IDF Features

We train using the TF/IDF features from above - as well as our labels.

_This will take a few minutes.  Please be patient._

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier

objective  = 'binary:logistic'
max_depth  = 5
num_round  = 1

xgb_estimator = XGBClassifier(objective=objective,
                              num_round=num_round,
                              max_depth=max_depth)

xgb_estimator.fit(df_tfidf, y_train)

# Calculate Test Metrics

In [ ]:
!ls -al ./data/test.csv

In [ ]:
X_test, y_test = load_dataset(path='./data/test.csv', sep=',', header=0)

In [ ]:
X_test.shape

In [ ]:
X_test.head(5)

In [ ]:
# sample_review_positive = """I loved it!  I will recommend this to everyone."""
# df_sample_review = pd.DataFrame([sample_review_positive], columns=['review_body'])
# df_sample_review

# Test our Model

## Convert raw test inputs into TF/IDF vectors

In [ ]:
np_tfidf_test = feature_transform_fn(df_text=X_test, column_name='review_body').fit_transform(X_test)
df_tfidf_test = pd.DataFrame(np_tfidf_test)
df_tfidf_test.shape

In [ ]:
preds_test = xgb_estimator.predict(df_tfidf_test)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, classification_report, confusion_matrix

print('Test Accuracy: ', accuracy_score(y_test, preds_test))
print('Test Precision: ', precision_score(y_test, preds_test, average=None))

In [ ]:
print(classification_report(y_test, preds_test))

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

df_cm_test = confusion_matrix(y_test, preds_test)
df_cm_test

In [ ]:
import itertools
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

def plot_conf_mat(cm, classes, title, cmap = plt.cm.Greens):
    print(cm)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
        horizontalalignment="center",
        color="black" if cm[i, j] > thresh else "black")

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')

# Plot non-normalized confusion matrix
plt.figure()
fig, ax = plt.subplots(figsize=(6,4))
plot_conf_mat(df_cm_test, classes=['Not Positive Sentiment', 'Positive Sentiment'], 
                          title='Confusion matrix')
plt.show()

In [ ]:
from sklearn import metrics

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

auc = round(metrics.roc_auc_score(y_test, preds_test), 4)
print('AUC is ' + repr(auc))

fpr, tpr, _ = metrics.roc_curve(y_test, preds_test)

plt.title('ROC Curve')
plt.plot(fpr, tpr, 'b',
label='AUC = %0.2f'% auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.1,1.1])
plt.ylim([-0.1,1.1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

# Save Model

In [ ]:
import os

import pickle as pkl

# See https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html
# Need to save with joblib or pickle.  `xgb.save_model()` does not save feature_names
model_dir  = './models/notebook/'

os.makedirs(model_dir, exist_ok=True)
model_path = os.path.join(model_dir, 'xgboost-model')

pkl.dump(xgb_estimator, open(model_path, 'wb'))

print('Wrote model to {}'.format(model_path))

# Restore Model 
This simulates restoring a model within an application.

In [ ]:
import pickle as pkl
import os

model_dir  = './models/notebook/'
model_path = os.path.join(model_dir, 'xgboost-model')

xgb_estimator = pkl.load(open(model_path, 'rb'))

type(xgb_estimator)

# Confirm the Predictions are OK from the Restored Model

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, classification_report, confusion_matrix

print('Test Accuracy: ', accuracy_score(y_test, preds_test))
print('Test Precision: ', precision_score(y_test, preds_test, average=None))